In [8]:
import tensorflow as tf
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab mnist.npz from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location
path = f"{getcwd()}/../Notebook-SimpleCNN/mnist.npz"

In [18]:
# GRADED FUNCTION: train_mnist_conv
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()

mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data(path=path)
    
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.05

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model fitting
history = model.fit(training_images, training_labels, epochs=20, callbacks= [callbacks])


Train on 60000 samples
Epoch 1/20
60000/60000 [==============================] - 85s 1ms/sample - loss: 0.1016 - accuracy: 0.9679
Epoch 2/20
60000/60000 [==============================] - 86s 1ms/sample - loss: 0.0357 - accuracy: 0.9886
Epoch 3/20
60000/60000 [==============================] - 87s 1ms/sample - loss: 0.0233 - accuracy: 0.9925
Epoch 4/20
60000/60000 [==============================] - 89s 1ms/sample - loss: 0.0169 - accuracy: 0.9947
Epoch 5/20
59936/60000 [============================>.] - ETA: 0s - loss: 0.0128 - accuracy: 0.9958
Reached 99.5% accuracy so cancelling training!
60000/60000 [==============================] - 90s 2ms/sample - loss: 0.0128 - accuracy: 0.9959


In [19]:
# Save the trained model as a Keras HDF5 file. 

saved_model_path = "./my_model.h5"

import time
saved_model_path = saved_model_path.format(int(time.time()))

model.save(saved_model_path)

# Use the tensorflow.js converter to convert the saved Keras model into JSON format.
!tensorflowjs_converter --input_format=keras {saved_model_path} ./